In [1]:
# This block of code uses several type definitions.
NumericalConstant = RealField(128)
DisplayConstant = RealField(16)
UnivariatePolynomial.<r> = PolynomialRing(NumericalConstant)

#
# Given a cubic corresponding to an elliptic curve along with a list of points on that elliptic curve,
# returns integral values a4 and a6 describing its short Weierstrass form along with a corresponding
# list of points on the new curve.
#
def compute_weierstrass_form(cubic, points):
    
    # Get 'a4' and 'a6' using a built-in SageMath function. By default, these values will be treated as
    # polynomials in 'x' and 'y', and their coefficients may be be rational. We fix this by taking the
    # constant coefficient of each and multiplying each by a power of 6, making sure to do a similar
    # multiplication for the coordinates of hte Weierstrass points later on.
    a4, a6 = WeierstrassForm(cubic)
    a4 = 6^4*a4.constant_coefficient(); a6 = 6^6*a6.constant_coefficient()
    
    # Get the transformation that must be applied to the points using the same built-in SageMath
    # function, then apply this transformation to each of the points and return.
    A, B, C = WeierstrassForm(cubic, transformation = True)
    X = 6^2*A/C^2; Y = 6^3*B/C^3
    weierstrass_points = [(
        X.substitute(x = point[0], y = point[1]),
        Y.substitute(x = point[0], y = point[1])
    ) for point in points]
    return (a4, a6, weierstrass_points)

#
# Given a list of possibly multivariate polynomials, returns a single polynomial such that the value of
# each of the original polynomials at every point is bounded by the returned polynomial evaluated at the
# absolute value of the largest coordinate of that point.
#
def compute_univariate_bound(polynomials):
    
    # Initialize a polynomial which will hold the returned bound, then enumerate through the given list
    # of polynomials.
    result = UnivariatePolynomial(0)
    for polynomial in polynomials:
        
        # For each given polynomial, compute a bound for it by summing up upper bounds for the absolute
        # values of each monomial.
        bound = UnivariatePolynomial(0)
        for monomial in polynomial.monomials():
            bound += abs(polynomial.monomial_coefficient(monomial))*r^monomial.degree()
            
        # Then construct a new polynomial which is the termwise maximum of 'result' and 'bound', and
        # reinitialize 'result' to this polynomial.
        new_result = UnivariatePolynomial(0)
        for i in range(max(result.degree(), bound.degree()) + 1):
            new_result += max(result.monomial_coefficient(r^i), bound.monomial_coefficient(r^i))*r^i
        result = new_result
        
    # Finally, return 'result', which now holds the termwise maximum of bounds for all of the given
    # polynomials.
    return result

#
# Represents a monoimal of the form 'coefficient*(r + offset)^power'. We will use such monomials to simplify
# more general polynomial bounds, which will be especially useful when dealing with bounds for Weil heights.
#
class BoundMonomial:
    
    #
    # Initializes a bound monomial with the given coefficient, power, and offset.
    #
    def __init__(self, coefficient, power, offset):
        self.coefficient = coefficient
        self.power = power
        self.offset = offset
        return
    
    #
    # Returns a string representation of this bound monomial.
    #
    def __str__(self):
        return str(DisplayValue(self.coefficient)) + "*(r + " + str(self.offset) + ")^" + str(self.power)
    
    #
    # Evaluates this bound monomial at the specified parameter.
    #
    def evaluate(self, parameter):
        return self.coefficient*(parameter + self.offset)^self.power

#
# Given a list of possibly multivariate polynomials, returns a bound monomial that can be used to bound
# all of the polynomials simultaneously.
#
def compute_univariate_bound_monomial(polynomials):
    
    # Start by obtaining a univariate bound for the given polynomials, and get its leading coefficient
    # and degree.
    bound = compute_univariate_bound(polynomials)
    coefficient = bound.leading_coefficient(); power = bound.degree()
    
    # Next, compute the required offset, which, for convenience, we restrict to be an integer. Initialize
    # this offset to zero, then enter a loop that increments the offset after each completion.
    offset = 0
    while True:
        
        # Expand the monomial corresponding to the current offset, and compare it termwise to the univariate
        # bound. If the monomial is larger, break; otherwise, continue with the next offset.
        expansion = coefficient*(r+offset)^power
        if all([expansion.monomial_coefficient(r^i) >= bound.monomial_coefficient(r^i) for i in range(power)]):
            break
        offset += 1
    
    # Finally, construct and return the relevant bound monomial.
    return BoundMonomial(coefficient, power, offset)

#
# Given a rational function in a finite number of variables, returns a bound monomial that can be used to
# bound the Naïve height of the rational function.
#
def compute_height_bound_monomial(rational_function):
    numerator = rational_function.numerator(); denominator = rational_function.denominator()
    return compute_univariate_bound_monomial([numerator, denominator])

#
# Represents a simplified bound for class numbers of discriminants associated to a particular
# elliptic curve.
#
class ClassNumberBound:
    
    #
    # Initializes a class number bound with the specified data, including information about the underlying
    # elliptic curve and bound monomials for several relevant constants.
    #
    def __init__(self, a4, a6, rank, torsion_size, regulator_bound, diameter_bound, delta_bound):
        self.a4 = a4; self.a6 = a6
        self.rank = rank
        self.torsion_size = torsion_size
        self.regulator_bound = regulator_bound
        self.diameter_bound = diameter_bound
        self.delta_bound = delta_bound
        return
    
    #
    # Returns the determinant 'd_E(u, v)' obtained from the elliptic curve corresponding to the specified
    # pair '(u, v)'.
    #
    def determinant(self, u, v):
        return v*(u^3 + self.a4*u*v^2 - self.a6*v^3)
    
    #
    # Returns true if the pair '(u, v)' is map-suitable.
    #
    def is_map_suitable(self, u, v):
        if u <= 0 or v <= 0:
            return False
        if 3*u^2 + self.a4*v^2 <= 0:
            return False
        if v*(u^3 + self.a4*u*v^2 - self.a6*v^3) <= 0:
            return False
        return True
    
    #
    # Returns the minimal value 'n' such that every pair of the form '(x + n*y, y)' with 'x' and 'y' positive
    # is map-suitable.
    #
    def shear_value(self):
        N = 1
        while 3*N^2 + self.a4 <= 0 or N^3 + self.a4*N - self.a6 <= 0:
            N *= 2
        N //= 2
        n = N
        while N > 1:
            N //= 2
            if 3*n^2 + self.a4 <= 0 or n^3 + self.a4*n - self.a6 <= 0:
                n += N
            else:
                n -= N
        if 3*n^2 + self.a4 <= 0 or n^3 + self.a4*n - self.a6 <= 0:
            n += 1
        return n
    
    #
    # Returns true if the pair '(u, v)' is kernel-suitable.
    #
    def is_kernel_suitable(self, u, v):
        if v <= 1:
            return False
        determinant = v*(u^3 + self.a4*u*v^2 - self.a6*v^3)
        torsion_points = EllipticCurve([self.a4, self.a6]).torsion_points()
        max_torsion_x = max([torsion_point[0] for torsion_point in torsion_points])
        if determinant - u*v <= max_torsion_x*v^2:
            return False
        return True
    
    #
    # Returns true if the pair '(u, v)' is 'epsilon'-bound-suitable.
    #
    def is_bound_suitable(self, u, v, epsilon):
        determinant = v*(u^3 + self.a4*u*v^2 - self.a6*v^3)
        if determinant <= (v*(u + v)*exp(2*(self.delta_bound + self.diameter_bound))/4)^(1/(1-epsilon)):
            return False
        if determinant >= (v^3*(u + v))^(1/(1-epsilon)):
            return False
        return True
    
    #
    # Returns true if the pair '(u, v)' is map-suitable, kernel-suitable, and 'epsilon'-bound suitable.
    #
    def is_suitable(self, u, v, epsilon):
        if not self.is_map_suitable(u, v):
            return False
        if not self.is_kernel_suitable(u, v):
            return False
        if not self.is_bound_suitable(u, v, epsilon):
            return False
        return True
    
    #
    # Given data 'parameters', 'u', 'v', and 'epsilon' such that '(u, v)' is map-suitable and 'epsilon'-bound
    # suitable, returns a lower bound for the class number 'h(-4*d_E(u, v))'.
    #
    def evaluate(self, u, v, epsilon):
        determinant = v*(u^3 + self.a4*u*v^2 - self.a6*v^3)
        T = (1/8)*log((4*determinant)^(1-epsilon)/(v*(u + v))) - (1/4)*self.delta_bound
        c = self.torsion_size/sqrt(self.regulator_bound)*pi^(self.rank/2)/(gamma(1+self.rank/2))
        return NumericalConstant(c*(T^(self.rank/2) - self.rank*sqrt(self.diameter_bound)*T^((self.rank-1)/2)))
    #
    # Returns a string representation of this class number bound.
    #
    def __str__(self):
        n = self.shear_value()
        
        P.<x, y> = PolynomialRing(DisplayConstant); var('epsilon')
        
        u = P(x + n*y); v = P(y)
        determinant = v*(u^3 + self.a4*u*v^2 - self.a6*v^3);
        
        torsion_points = EllipticCurve([self.a4, self.a6]).torsion_points()
        max_torsion_x = max([torsion_point[0] for torsion_point in torsion_points])
        kernel_suitable_polynomial = determinant - u*v - max_torsion_x*v^2
        
        bound_suitable_lower = P(v*(u + v)*exp(2*(self.delta_bound + self.diameter_bound))/4)
        bound_suitable_upper = P(v^3*(u + v))
        
        hypothesis = "x > 0 and y > 1 and\n" + str(kernel_suitable_polynomial) + " > 0 and\n" + str(bound_suitable_lower) + " < (" + str(determinant) + ")^(1 - epsilon) < " + str(bound_suitable_upper)
        
        T8 = "log((" + str(4*determinant) + ")^(1-epsilon)/(" + str(P(v*(u + v))) + ")) - " + str(DisplayConstant(2*self.delta_bound))
        c = self.torsion_size/sqrt(self.regulator_bound)*pi^(self.rank/2)/(gamma(1+self.rank/2))
        conclusion = "h(" + str(-4*determinant) + ") >= " + str(DisplayConstant(c/8^(self.rank/2))) + "*T^(" + str(self.rank/2) + ") - " + str(DisplayConstant(c*self.rank*sqrt(self.diameter_bound)/8^((self.rank-1)/2))) + "*T^(" + str((self.rank-1)/2) + ")"
        
        definition = "T := " + T8
        
        reset('x y epsilon')
        
        return "if " + hypothesis + "\n\nthen\n\n" + conclusion + "\n\nwhere\n\n" + definition

#
# Represents a simplified uniform bound for class numbers of discriminants associated to a particular
# family of elliptic curves.
#
class ClassNumberBoundFamily:
    
    #
    # Initializes a class number bound family with the specified data, including information about the
    # underlying elliptic curve family and bound monomials for several relevant constants.
    #
    def __init__(self, a4, a6, rank, torsion_size, offset, regulator_bound, diameter_bound, delta_bound):
        self.a4 = a4; self.a6 = a6
        self.rank = rank
        self.torsion_size = torsion_size
        self.offset = offset
        self.regulator_bound = regulator_bound
        self.diameter_bound = diameter_bound
        self.delta_bound = delta_bound
        return
    
    #
    # Returns the class number bound obtained from the elliptic curve corresponding to the specified
    # parameters.
    #
    def specialize(self, parameters):
        parameter_bound = max([abs(parameter) for parameter in parameters])
        return ClassNumberBound(
            self.a4(parameters), self.a6(parameters), self.rank, self.torsion_size,
            NumericalConstant(self.regulator_bound.evaluate(log(parameter_bound + self.offset))),
            NumericalConstant(self.diameter_bound.evaluate(log(parameter_bound + self.offset))),
            NumericalConstant(self.delta_bound.evaluate(log(parameter_bound + self.offset)))
        )
            

#
# Given polynomials 'a4' and 'a6' describing a family of elliptic curves in short Weierstrass form, a list of
# linearly independent points on these elliptic curves, and a lower bound for the size of the torsion
# subgroup of the elliptic curve, returns a corresponding class number bound family.
#
def compute_class_number_lower_bound_family(a4, a6, points, torsion_size):
    # Get a lower bound for the rank of the elliptic curve.
    rank = len(points)
    
    # Get monomial bounds for the heights of the discriminant, the j-invariant, and the x-coordinates of all
    # of the given points, then find the maximum offset used in all of these monomial bounds.
    discriminant = -16*(4*a4^3+27*a6^2)
    discriminant_height_bound = compute_height_bound_monomial(discriminant)
    j_invariant = 1728*4*a4^3/discriminant
    j_invariant_height_bound = compute_height_bound_monomial(j_invariant)
    x_height_bounds = [compute_height_bound_monomial(point[0]) for point in points]
    max_offset = max(
        discriminant_height_bound.offset,
        j_invariant_height_bound.offset,
        max([x_height_bound.offset for x_height_bound in x_height_bounds])
    )
    
    # Use Hadamard's inequality to get a monomial bound for the regulator.
    regulator_polynomial = product([UnivariatePolynomial(
        (
            (1/12)*log(j_invariant_height_bound.coefficient) +
            (1/12)*log(discriminant_height_bound.coefficient) +
            (1/2)*log(x_height_bounds[i].coefficient) + 1.07
        ) + (
            (1/12)*j_invariant_height_bound.power +
            (1/12)*discriminant_height_bound.power +
            (1/2)*x_height_bounds[i].power
        )*r
    ) for i in range(rank)])
    regulator_bound = compute_univariate_bound_monomial([regulator_polynomial])
    
    # Use the triangle inequality to get a monomial bound for the diameter.
    diameter_polynomial = 2*sum([UnivariatePolynomial(
        (
            (1/12)*log(j_invariant_height_bound.coefficient) +
            (1/12)*log(discriminant_height_bound.coefficient) +
            (1/2)*log(x_height_bounds[i].coefficient) + 1.07
        ) + (
            (1/12)*j_invariant_height_bound.power +
            (1/12)*discriminant_height_bound.power +
            (1/2)*x_height_bounds[i].power
        )*r
    ) for i in range(rank)])
    diameter_bound = compute_univariate_bound_monomial([diameter_polynomial])
    
    # Compute a monomial bound for the value 'delta(E)' defined in our paper. We use the
    # 'compute_univariate_bound_monomial' function for convenience, though it is a bit overkill.
    delta_bound = compute_univariate_bound_monomial([UnivariatePolynomial((
        (1/8)*log(j_invariant_height_bound.coefficient) + (1/12)*log(discriminant_height_bound.coefficient) + 5/3
    ) + (
        (1/8)*log(j_invariant_height_bound.power) + (1/12)*log(discriminant_height_bound.power) + 5/3
    )*r)])
    
    # Assemble all of the data into a class number bound.
    return ClassNumberBoundFamily(a4, a6, rank, torsion_size, max_offset, regulator_bound, diameter_bound, delta_bound)


In [13]:
#
# This cell contains the example we give in our paper for torsion structure Z/2Z. The cubic defining the
# elliptic curve as well as the formulas for the linearly independent points  are taken from citation [9]
# in our paper:
# N. Elkies and Z. Klagsbrun, New rank records for elliptic curves having rational torsion, Preprint (2020).
#

# Type definitions used for this example.
Polynomial2.<t1, t2> = PolynomialRing(ZZ)
RationalFunction2 = Polynomial2.fraction_field()
EllipticCurve2.<x, y> = PolynomialRing(RationalFunction2)

# Construct the cubic that defines this elliptic curve family as in [9].
B1 = Polynomial2((t1^2+t1-8)*t2+(-t1+2))
B2 = Polynomial2((t1^2-t1-8)*t2-(t1+2))
B3 = Polynomial2((t1^2-t1-8)*t2+(t1^2+t1-10))
B4 = Polynomial2((t1^2+t1-8)*t2-(t1^2-t1-10))
B5 = Polynomial2((t1^2-7*t1+8)*t2+(-t1^2+t1+2))
B6 = Polynomial2((t1^2+7*t1+8)*t2-(-t1^2-t1+2))
B7 = Polynomial2((t1^2+5*t1+8)*t2+(t1^2+3*t1+2))
B8 = Polynomial2((t1^2-5*t1+8)*t2-(t1^2-3*t1+2))
A = Polynomial2((t1^8-18*t1^6+163*t1^4-1152*t1^2+4096)*t2^4+(3*t1^7-35*t1^5-120*t1^3+1536*t1)*t2^3+(t1^8-13*t1^6+32*t1^4-152*t1^2+1536)*t2^2+(t1^7+3*t1^5-156*t1^3+672*t1)*t2+(3*t1^6-33*t1^4+112*t1^2-80))
B = B1*B2*B3*B4*B5*B6*B7*B8
cubic = EllipticCurve2(y^2 - (x^3 + 2*A*x^2 + B*x))

# Construct the linearly independent points on the elliptic curve as in [9].
x1 = -B1*B2*B3*B6
x2 = -B1*B2*B4*B5
x3 = 4*B1*B2*B5*B6
x4 = B1*B3*B4*B6
x5 = -B1*B3*B4*B7
x6 = B1*B3*B4*B8
x7 = B1*B3*B5*B6
x8 = -B1*B5*B6*B7
var('m')
y1 = Polynomial2(solve(m^2 - (x1^3+2*A*x1^2+B*x1), m)[0].rhs())
y2 = Polynomial2(solve(m^2 - (x2^3+2*A*x2^2+B*x2), m)[0].rhs())
y3 = Polynomial2(solve(m^2 - (x3^3+2*A*x3^2+B*x3), m)[0].rhs())
y4 = Polynomial2(solve(m^2 - (x4^3+2*A*x4^2+B*x4), m)[0].rhs())
y5 = Polynomial2(solve(m^2 - (x5^3+2*A*x5^2+B*x5), m)[0].rhs())
y6 = Polynomial2(solve(m^2 - (x6^3+2*A*x6^2+B*x6), m)[0].rhs())
y7 = Polynomial2(solve(m^2 - (x7^3+2*A*x7^2+B*x7), m)[0].rhs())
y8 = Polynomial2(solve(m^2 - (x8^3+2*A*x8^2+B*x8), m)[0].rhs())
reset('m')
points = [(x1,y1),(x2,y2),(x3,y3),(x4,y4),(x5,y5),(x6,y6),(x7,y7),(x8,y8)]

# Compute the short Weierstrass form of the elliptic curve family and the linearly independent points.
a4, a6, weierstrass_points = compute_weierstrass_form(cubic, points)

# Uncomment this block to print 'a4', 'a6', and the entries in 'weierstrass_points'.
# print("a4 = " + str(a4))
# print("")
# print("a6 = " + str(a6))
# print("")
# for i in range(len(weierstrass_points)):
#     print("weierstrass_points[" + str(i) + "] = " + str(weierstrass_points[i]))
#     print("")

# Compute a 'ClassNumberBoundFamily' instance for this family, which can be used to obtain explicit numerical
# lower bounds for class numbers.
class_number_bound_family2 = compute_class_number_lower_bound_family(a4, a6, weierstrass_points, 2)

# Uncomment this block and replace 't1' and 't2' with integral constants to print the class number lower bound
# obtained from the elliptic curve corresponding to the specified parameters.
# class_number_bound2 = class_number_bound_family2.specialize([t1, t2])
# print(str(class_number_bound2))

KeyboardInterrupt: 

In [6]:
#
# This cell contains the example we give in our paper for torsion structure Z/3Z. The cubic defining the
# elliptic curve as well as the formulas for the linearly independent points  are taken from citation [22]
# in our paper:
# L. Kulesz, Families of elliptic curves of high rank with nontrivial torsion group over Q. Acta Arith. 108(2003), 339-356.
#

# Type definitions used for this example.
Polynomial3.<t1, t2, t3> = PolynomialRing(ZZ)
RationalFunction3 = Polynomial3.fraction_field()
EllipticCurve3.<x, y> = PolynomialRing(RationalFunction3)

# Construct the cubic that defines this elliptic curve family as in [22], making sure that its coefficients
# are polynomials with integral coefficients. Use the line below to specify which of the three parameters to
# vary and what to set the other variables to.
x1 = t1; x2 = 4; x3 = 6
def g2(xi):
    return -(xi^2+3)^3/(4*(xi-1)^2*(xi+1)^2)
X1 = g2(x1); X2 = g2(x2); X3 = g2(x3)
c3 = -(X1+X2+X3); c2 = X1*X2+X1*X3+X2*X3; c1 = -X1*X2*X3
d1 = c3/2; d0 = (c2-d1^2)/2
r1 = 2*d1*d0 - c1; r2 = d0
g = lcm([r1.denominator(), (r2^2).denominator()])
cubic = EllipticCurve3(g*(y^2 - r1*x^3 - r2^2*(x+1)^2))

# Construct the linearly independent points on the elliptic curve as in [22].
SolvePolynomial3.<m> = PolynomialRing(Polynomial3)
SolveRationalFunction3 = SolvePolynomial3.fraction_field()
F2 = SolveRationalFunction3(m^3/(m+1)^2)
product = SolveRationalFunction3((F2-g2(x1))*(F2-g2(x2))*(F2-g2(x3)))
product_num = product.numerator()
var('M')
roots = solve(product_num(M), M)
points = []
for i in range(6):
    x_coord = roots[i].rhs()
    y_coord = solve(M^2 - r1*x_coord^3 - r2^2*(x_coord+1)^2, M)[0].rhs()
    points.append((RationalFunction3(r1*x_coord), RationalFunction3(r1*(r2*(x_coord+r1)+y_coord))))
reset('M')

# Compute the short Weierstrass form of the elliptic curve family and the linearly independent points.
a4, a6, weierstrass_points = compute_weierstrass_form(cubic, points)

# Uncomment this block to print 'a4', 'a6', and the entries in 'weierstrass_points'.
# print("a4 = " + str(a4))
# print("")
# print("a6 = " + str(a6))
# print("")
# for i in range(len(weierstrass_points)):
#     print("weierstrass_points[" + str(i) + "] = " + str(weierstrass_points[i]))
#     print("")

# Compute a 'ClassNumberBoundFamily' instance for this family, which can be used to obtain explicit numerical
# lower bounds for class numbers.
class_number_bound_family3 = compute_class_number_lower_bound_family(a4, a6, weierstrass_points, 3)

# Uncomment this block and replace 't1', 't2', and 't3' with integral constants to print the class number lower bound
# obtained from the elliptic curve corresponding to the specified parameters.
# class_number_bound2 = class_number_bound_family3.specialize([t1, t2, t3])
# print(str(class_number_bound3))

foo
foo
foo
foo
foo
foo
bar
-36212718300376644285164585765287230469789520363520000000000000000*t1^80 + 13879065426518667475771956144478742059063377699589324800000000000000*t1^78 - 2051679857665931023276951001944241919366520207598715666432000000000000*t1^76 + 164163900500789746300772677017705087476023147095871305579233280000000000*t1^74 - 7702894260010983864556792761832443142454680913548334078539019059200000000*t1^72 + 137799270814666391610921683405777679250152066609031968759445507276800000000*t1^70 - 5015407575937735279009311845803941530518790661486327327169796112384000000000*t1^68 + 62435667858726407636576589535153423867781591384741300259678231252172800000000*t1^66 - 453026055464624180846393218356495223007756457914908613985082344996864000000000*t1^64 + 2954774591184870803887793458331704254278988637049301112287671362152038400000000*t1^62 - 20301656827808200152820380619397331584049920439167912741555599382151168000000000*t1^60 + 12602202902543543298513631285949599449644910988668553312440

In [25]:
#
# This cell contains the example we give in our paper for torsion structure Z/4Z. The cubic defining the
# elliptic curve as well as the formulas for the linearly independent points  are taken from citation [20]
# in our paper:
# S. Kihara, On the  rank of the elliptic curves with a rational point of order 4, II, Proc. Japan Acad. Ser. AMath. Sci. 80(2004), 158–159.
#

# Type definitions used for this example.
Polynomial4.<t1> = PolynomialRing(ZZ)
RationalFunction4 = Polynomial4.fraction_field()
EllipticCurve4.<x, y> = PolynomialRing(RationalFunction4)

# Construct the cubic that defines this elliptic curve family as in [20].
o = Polynomial4(-(t1^2-1)*(12*t1^11-219*t1^10+1699*t1^9-7248*t1^8+21004*t1^7-45434*t1^6+72862*t1^5-90128*t1^4+77496*t1^3-46283*t1^2+10095*t1-768))
s = Polynomial4(3*t1^13-128*t1^12+1185*t1^11-5018*t1^10+13628*t1^9-27704*t1^8+44162*t1^7-63956*t1^6+84827*t1^5-100976*t1^4+92061*t1^3-52802*t1^2+10662*t-552)
w = Polynomial4(-21*t1^13+330*t1^12+2117*t1^11+8532*t1^10-24566*t1^9+51764*t1^8-83474*t1^7+99728*t1^6-92921*t1^5+63962*t1^4-39209*t1^3+21228*t1^2-8828*t1+984)
p = Polynomial4(-6*t1^13+33*t1^12-155*t1^11+1911*t1^10-11855*t1^9+43046*t1^8-106778*t1^7+187562*t1^6-236720*t1^5+215945*t1^4-128363*t1^3+53439*t1^2-9179*t1+336)
q = Polynomial4(30*t1^13-443*t1^12+2589*t1^11-9725*t1^10+28685*t1^9-67490*t1^8+130502*t1^7-199886*t1^6+238460*t1^5-201395*t1^4+109245*t1^3-15317*t1^2-7239*t1+1200)
n = Polynomial4(-15*t1^13+138*t1^12-259*t1^11-2160*t1^10+17250*t1^9-66700*t1^8+165618*t1^7-291384*t1^6+370045*t1^5-325350*t1^4+195345*t1^3-54248*t1^2+5424*t1+120)
a = Polynomial4((2*o^2-s^2-w^2)/2)
b = Polynomial4(s^2*w^2 + s^2*o^2 + w^2*o^2 - 3*o^4)
cubic = EllipticCurve4(y^2 - (x*(x^2 + (2*a^2 + 2*b)*x + (a^2-b^2)^2)))

# Construct the linearly independent points on the elliptic curve as in [20].
P1 = (o, s); P2 = (o, w); P3 = (s, p); P4 = (w, q); P5 = (p, n)
X1 = (a^2-b^2)*P1[1]^2/P1[0]^2
Y1 = (a^2-b)*P1[1]*(((b)+t)*P1[0]^2+t*P1[1]^2)/P1[0]^3
X2 = (a^2-b^2)*P2[1]^2/P2[0]^2
Y2 = (a^2-b)*P2[1]*(((b)+a)*P2[0]^2+a*P2[1]^2)/P2[0]^3
X3 = (a^2-b^2)*P3[1]^2/P3[0]^2
Y3 = (a^2-b)*P3[1]*(((b)+a)*P3[0]^2+a*P3[1]^2)/P3[0]^3
X4 = (a^2-b^2)*P4[1]^2/P4[0]^2
Y4 = (a^2-b)*P4[1]*(((b)+a)*P4[0]^2+a*P4[1]^2)/P4[0]^3
X5 = (a^2-b^2)*P5[1]^2/P5[0]^2
Y5 = (a^2-b)*P5[1]*(((b)+a)*P5[0]^2+a*P5[1]^2)/P5[0]^3
points = [(X1,Y1), (X2,Y2), (X3,Y3), (X4,Y4), (X5,Y5)]

# Compute the short Weierstrass form of the elliptic curve family and the linearly independent points.
a4, a6, weierstrass_points = compute_weierstrass_form(cubic, points)

# Uncomment this block to print 'a4', 'a6', and the entries in 'weierstrass_points'.
# print("a4 = " + str(a4))
# print("")
# print("a6 = " + str(a6))
# print("")
# for i in range(len(weierstrass_points)):
#     print("weierstrass_points[" + str(i) + "] = " + str(weierstrass_points[i]))
#     print("")

# Compute a 'ClassNumberBoundFamily' instance for this family, which can be used to obtain explicit numerical
# lower bounds for class numbers.
class_number_bound_family4 = compute_class_number_lower_bound_family(a4, a6, weierstrass_points, 4)

# Uncomment this block and replace 't1' with an integral constant to print the class number lower bound
# obtained from the elliptic curve corresponding to the specified parameters.
# class_number_bound4 = class_number_bound_family4.specialize([t1])
# print(str(class_number_bound4))

(t1 + t2) * m

In [3]:
#
# This cell contains the example we give in our paper for torsion structure Z/5Z. The cubic defining the
# elliptic curve as well as the formulas for the linearly independent points  are taken from citation [22]
# in our paper:
# L. Kulesz, Families of elliptic curves of high rank with nontrivial torsion group over Q. Acta Arith. 108(2003), 339-356.
#

# Type definitions used for this example.
Polynomial5.<t1> = PolynomialRing(ZZ)
RationalFunction5 = Polynomial5.fraction_field()
EllipticCurve5.<x, y> = PolynomialRing(RationalFunction5)

# Construct the cubic that defines this elliptic curve family as in [22].
b = RationalFunction5(-((3*t1^2+6*t1+4)*(t1^2+6*t1+12))/((t1-2)^2*(t1+2)^2))
w = RationalFunction5(-(8+8*t1+t1^2)/((t1-2)*(t1+2)))
v = RationalFunction5(-(t1^2+6*t1+12)/((t1-2)*(t1+2)))
cubic = EllipticCurve5(((t1-2)^2*(t1+2)^2)*(y^2 + (1-b)*x*y - b*y - x^3 + b*x^2))

# Construct the linearly independent points on the elliptic curve as in [22].
points = [(-1,w), (v,v)]

# Compute the short Weierstrass form of the elliptic curve family and the linearly independent points.
a4, a6, weierstrass_points = compute_weierstrass_form(cubic, points)

# Uncomment this block to print 'a4', 'a6', and the entries in 'weierstrass_points'.
# print("a4 = " + str(a4))
# print("")
# print("a6 = " + str(a6))
# print("")
# for i in range(len(weierstrass_points)):
#     print("weierstrass_points[" + str(i) + "] = " + str(weierstrass_points[i]))
#     print("")

# Compute a 'ClassNumberBoundFamily' instance for this family, which can be used to obtain explicit numerical
# lower bounds for class numbers.
class_number_bound_family5 = compute_class_number_lower_bound_family(a4, a6, weierstrass_points, 5)

# Uncomment this block and replace 't1' with an integral constant to print the class number lower bound
# obtained from the elliptic curve corresponding to the specified parameters.
# class_number_bound5 = class_number_bound_family5.specialize([t1])
# print(str(class_number_bound5))

In [ ]:
#
# This cell contains the example we give in our paper for torsion structure Z/6Z. The cubic defining the
# elliptic curve as well as the formulas for the linearly independent points  are taken from citation [22]
# in our paper:
# L. Kulesz, Families of elliptic curves of high rank with nontrivial torsion group over Q. Acta Arith. 108(2003), 339-356.
#

# Type definitions used for this example.
Polynomial6.<t1> = PolynomialRing(ZZ)
RationalFunction6 = Polynomial6.fraction_field()
EllipticCurve6.<x, y> = PolynomialRing(RationalFunction6)

# Construct the cubic that defines this elliptic curve family as in [22].
c = RationalFunction6(4*(t1-1)*(-2*t1+1+2*t1^2)/(5 - 8*t1 + 4*t1^4))
cubic = EllipticCurve6((5 - 8*t1 + 4*t1^4)*(y^2 + (1-c)*x*y + (-c-c^2)*y - x^3 - (-c-c^2)*x^2))

# Construct the linearly independent points on the elliptic curve as in [22].
x1 = RationalFunction6(-c(; x2 = RationalFunction6(c*t1)
var('M')
y1 = RationalFunction6(solve(m^2 + (1-c)*x1*M + (-c-c^2)*M - x1^3 - (-c-c^2)*x1^2, M)[0].rhs())
y2 = RationalFunction6(solve(m^2 + (1-c)*x2*M + (-c-c^2)*M - x2^3 - (-c-c^2)*x2^2, M)[0].rhs())
reset('M')
points = [(x1,y1), (x2,y2)]

# Compute the short Weierstrass form of the elliptic curve family and the linearly independent points.
a4, a6, weierstrass_points = compute_weierstrass_form(cubic, points)

# Uncomment this block to print 'a4', 'a6', and the entries in 'weierstrass_points'.
# print("a4 = " + str(a4))
# print("")
# print("a6 = " + str(a6))
# print("")
# for i in range(len(weierstrass_points)):
#     print("weierstrass_points[" + str(i) + "] = " + str(weierstrass_points[i]))
#     print("")

# Compute a 'ClassNumberBoundFamily' instance for this family, which can be used to obtain explicit numerical
# lower bounds for class numbers.
class_number_bound_family6 = compute_class_number_lower_bound_family(a4, a6, weierstrass_points, 6)

# Uncomment this block and replace 't1' with an integral constant to print the class number lower bound
# obtained from the elliptic curve corresponding to the specified parameters.
# class_number_bound6 = class_number_bound_family6.specialize([t1])
# print(str(class_number_bound6))

In [ ]:
#
# This cell contains the example we give in our paper for torsion structure Z/7Z. The cubic defining the
# elliptic curve as well as the formulas for the linearly independent points  are taken from citation [22]
# in our paper:
# L. Kulesz, Families of elliptic curves of high rank with nontrivial torsion group over Q. Acta Arith. 108(2003), 339-356.
#

# Type definitions used for this example.
Polynomial7.<t1> = PolynomialRing(ZZ)
RationalFunction7 = Polynomial7.fraction_field()
EllipticCurve7.<x, y> = PolynomialRing(RationalFunction7)

# Construct the cubic that defines this elliptic curve family as in [22].
d = RationalFunction7((-2*(t1-3))/(3+t1^2))
b = RationalFunction7(d^3-d^2)
c = RationalFunction7(d^2-d)
cubic = EllipticCurve7((3+t1^2)^2*(y^2 + (1-c)*x*y - b*y - x^3 + b*x^2))

# Construct the linearly independent points on the elliptic curve as in [22].
x1 = RationalFunction7((-2*(t1-1)*(t1+3)*(t1+1)*(t1-3)^2)/((3+t1^2)^3))
var('M')
y1 = RationalFunction7(solve(M^2 + (1-c)*x1*M - b*M - x1^3 + b*x1^2, M)[0].rhs())
reset('M')
points = [(x1,y1)]

# Compute the short Weierstrass form of the elliptic curve family and the linearly independent points.
a4, a6, weierstrass_points = compute_weierstrass_form(cubic, points)

# Uncomment this block to print 'a4', 'a6', and the entries in 'weierstrass_points'.
# print("a4 = " + str(a4))
# print("")
# print("a6 = " + str(a6))
# print("")
# for i in range(len(weierstrass_points)):
#     print("weierstrass_points[" + str(i) + "] = " + str(weierstrass_points[i]))
#     print("")

# Compute a 'ClassNumberBoundFamily' instance for this family, which can be used to obtain explicit numerical
# lower bounds for class numbers.
class_number_bound_family7 = compute_class_number_lower_bound_family(a4, a6, weierstrass_points, 7)

# Uncomment this block and replace 't1' with an integral constant to print the class number lower bound
# obtained from the elliptic curve corresponding to the specified parameters.
# class_number_bound7 = class_number_bound_family7.specialize([t1])
# print(str(class_number_bound7))

In [ ]:
#
# This cell contains the example we give in our paper for torsion structure Z/8Z. The cubic defining the
# elliptic curve as well as the formulas for the linearly independent points  are taken from citation [8]
# in our paper:
# A. Dujella and J.C. Peral, Elliptic curves with torsion group Z/8Z or Z/2Z×Z/6Z, Contemp. Math.649(2015), 47-62.
#

# Type definitions used for this example.
Polynomial8.<t1> = PolynomialRing(ZZ)
RationalFunction8 = Polynomial8.fraction_field()
EllipticCurve8.<x, y> = PolynomialRing(RationalFunction8)

# Construct the cubic that defines this elliptic curve family as in [8].
a = RationalFunction8(500246412961 - 2069985157080*t1 + 3162080774436*t1^2 - 2895517882032*t1^3 + 1873181389706*t1^4 - 906769167048*t1^5 + 333391978480*t1^6 - 93284915496*t1^7+ 19860033555*t1^8 - 3216721224*t1^9 + 396423280*t1^10 - 37179432*t1^11+2648426*t1^12 - 141168*t1^13 + 5316*t1^14 - 120*t1^15 + t1^16)
b = RationalFunction8(256*(-6 + t1)^4*t1^4*(-29 + 6*t1)^4*(841 - 522*t1 + 137*t1^2 - 18*t1^3 + t1^4)^4)
cubic = EllipticCurve7(y^2 - x^3 - a*x^2 - b*x)

# Construct the linearly independent points on the elliptic curve as in [8].
x1 = RationalFunction8(64*(-6+t1)^2*t1^2*(-29+6*t1)^2*(-29+t1^2)^2*(29-12*t1+t1^2)^2*(841-522*t1+137*t1^2-18*t1^3+t1^4)^4/((707281-292668*t1-200158*t1^2 + 168432*t1^3 - 46685*t1^4 + 5808*t1^5 - 238*t1^6 -12*t1^7 + t1^8)^2))
x2 = RationalFunction8((-6 + t1)^2*t1^2*(-29 + 6*t1)^2*(87 - 29*t1 + 3*t1^2)^2*(2523 - 1914*t1 + 541*t1^2 - 66*t1^3 + 3*t1^4)^2 / (4*(29 - 12*t1 + t1^2)^2))
var('M')
y1 = RationalFunction8(solve(M^2 - x1^3 - a*x1^2 - b*x1, M)[0].rhs())
y2 = RationalFunction8(solve(M^2 - x2^3 - a*x2^2 - b*x2, M)[0].rhs())
reset('M')
points = [(x1,y1),(x2,y2)]

# Compute the short Weierstrass form of the elliptic curve family and the linearly independent points.
a4, a6, weierstrass_points = compute_weierstrass_form(cubic, points)

# Uncomment this block to print 'a4', 'a6', and the entries in 'weierstrass_points'.
# print("a4 = " + str(a4))
# print("")
# print("a6 = " + str(a6))
# print("")
# for i in range(len(weierstrass_points)):
#     print("weierstrass_points[" + str(i) + "] = " + str(weierstrass_points[i]))
#     print("")

# Compute a 'ClassNumberBoundFamily' instance for this family, which can be used to obtain explicit numerical
# lower bounds for class numbers.
class_number_bound_family8 = compute_class_number_lower_bound_family(a4, a6, weierstrass_points, 8)

# Uncomment this block and replace 't1' with an integral constant to print the class number lower bound
# obtained from the elliptic curve corresponding to the specified parameters.
# class_number_bound8 = class_number_bound_family8.specialize([t1])
# print(str(class_number_bound8))

In [ ]:
#
# This cell contains the example we give in our paper for torsion structure Z/2Z x Z/2Z. The cubic defining the
# elliptic curve as well as the formulas for the linearly independent points are taken from citation [19]
# in our paper:
# S. Kihara, On the rank of elliptic curves with three rational points of order 2. III, Proc. Japan Acad. Ser. AMath. Sci.80(2004) 13-14.
#

# Type definitions used for this example.
Polynomial2x2.<t1> = PolynomialRing(ZZ)
RationalFunction2x2 = Polynomial2x2.fraction_field()
EllipticCurve2x2.<x, y> = PolynomialRing(RationalFunction2x2)

# Construct the cubic that defines this elliptic curve family as in [19].
A = Polynomial2x2((t1+1)*(2*t1^8-14*t1^7+146*t1^6-473*t1^5+674*t1^4-473*t1^3+146*t1^2-14*t1+2))
B = Polynomial2x2((t1-2)*(2*t1^8-2*t1^7+104*t1^6-221*t1^5+149*t1^4-35*t1^3-7*t1^2+16*t1-4))
C = Polynomial2x2((2*t1-1)*(4*t1^8-16*t1^7+7*t1^6+35*t1^5-149*t1^4+221*t1^3-104*t1^2+2*t1-2))
D = Polynomial2x2(2*t1^9+18*t1^8-144*t1^7+411*t1^6-477*t1^5+225*t1^4-75*t1^3+72*t1^2-36*t1+2)
E = Polynomial2x2((t1-2)^3*(t1+1)^3*(2*t1-1)^3)
F = Polynomial2x2(2*t1^9-36*t1^8+72*t1^7-75*t1^6+225*t1^5-477*t1^4+411*t1^3-144*t1^2+18*t1+2)
a = RationalFunction2x2((A^4+B^4+C^4)/(A^2*B^2+A^2*C^2+B^2*C^2))
cubic = EllipticCurve2x2((A^2*B^2+A^2*C^2+B^2*C^2)^3*(y^2 - (x*(x-4*a-8)*(x-4*a^2-4*a+8))))

# Construct the linearly independent points on the elliptic curve as in [19].
x1 = RationalFunction2x2((2*B^2-a*A^2-a*C^2)^2/(B*C)^2)
x2 = RationalFunction2x2((2*C^2-a*B^2-a*A^2)^2/(C*A)^2)
x3 = RationalFunction2x2((2*A^2-a*C^2-a*B^2)^2/(A*B)^2)
x4 = RationalFunction2x2((2*E^2-a*D^2-a*F^2)^2/(E*F)^2)
x5 = RationalFunction2x2((2*F^2-a*E^2-a*D^2)^2/(F*D)^2)
x6 = RationalFunction2x2((2*D^2-a*F^2-a*E^2)^2/(D*E)^2)
y1 = RationalFunction2x2((a^2-4)*(C^4-B^4)*(2*B^2-a*A^2-a*C^2)/(B*C)^3)
y2 = RationalFunction2x2((a^2-4)*(A^4-C^4)*(2*C^2-a*B^2-a*A^2)/(C*A)^3)
y3 = RationalFunction2x2((a^2-4)*(B^4-A^4)*(2*A^2-a*C^2-a*B^2)/(A*B)^3)
y4 = RationalFunction2x2((a^2-4)*(F^4-E^4)*(2*E^2-a*D^2-a*F^2)/(E*F)^3)
y5 = RationalFunction2x2((a^2-4)*(D^4-F^4)*(2*F^2-a*E^2-a*D^2)/(F*D)^3)
y6 = RationalFunction2x2((a^2-4)*(E^4-D^4)*(2*D^2-a*F^2-a*E^2)/(D*E)^3)
points = [(x1,y1),(x2,y2)]

# Compute the short Weierstrass form of the elliptic curve family and the linearly independent points.
a4, a6, weierstrass_points = compute_weierstrass_form(cubic, points)

# Uncomment this block to print 'a4', 'a6', and the entries in 'weierstrass_points'.
# print("a4 = " + str(a4))
# print("")
# print("a6 = " + str(a6))
# print("")
# for i in range(len(weierstrass_points)):
#     print("weierstrass_points[" + str(i) + "] = " + str(weierstrass_points[i]))
#     print("")

# Compute a 'ClassNumberBoundFamily' instance for this family, which can be used to obtain explicit numerical
# lower bounds for class numbers.
class_number_bound_family2x2 = compute_class_number_lower_bound_family(a4, a6, weierstrass_points, 4)

# Uncomment this block and replace 't1' with an integral constant to print the class number lower bound
# obtained from the elliptic curve corresponding to the specified parameters.
# class_number_bound2x2 = class_number_bound_family2x2.specialize([t1])
# print(str(class_number_bound2x2))

In [ ]:
#
# This cell contains the example we give in our paper for torsion structure Z/2Z x Z/4Z. The cubic defining the
# elliptic curve as well as the formulas for the linearly independent points are taken from citation [7]
# in our paper:
# A. Dujella and J.C. Peral, High-rank elliptic curves with torsion Z/2Z×Z/4Z induced by Diophantine triples, LMS J. Comput. Math. (1)17(2014), 282-288.
#

# Type definitions used for this example.
Polynomial2x4.<t1> = PolynomialRing(ZZ)
RationalFunction2x4 = Polynomial2x4.fraction_field()
EllipticCurve2x4.<x, y> = PolynomialRing(RationalFunction2x4)

# Construct the cubic that defines this elliptic curve family as in [19].
A = Polynomial2x4(2*(87671889*t1^24 + 854321688*t1^23 + 3766024692*t1^22 + 9923033928*t1^21+ 17428851514*t1^20 + 21621621928*t1^19 + 19950275060*t1^18+ 15200715960*t1^17 + 11789354375*t1^16 + 10470452464*t1^15 + 8925222696*t1^14 + 5984900048*t1^13 + 2829340620*t1^12 + 820299856*t1^11 + 59930952*t1^10- 66320528*t1^9 - 35768977*t1^8 - 9381000*t1^7 - 1017244*t1^6 + 262760*t1^5 + 159130*t1^4 + 41096*t1^3 + 6468*t1^2 + 600*t1 + 25))
B = Polynomial2x4((t1^2 -2*t1-1)^2*(69*t1^4 +148*t1^3 +78*t1^2 +4*t1+1)^2*(13*t1^2 -2*t1-1)^2*(9*t1^4 +28*t1^3 +18*t1^2 +4*t1+1)^2*(11*t1^4 +12*t1^3 +2*t1^2 -4*t1-1)^2*(9*t1^2 +14*t1+7)^2*(31*t1^4 +52*t1^3 +22*t1^2 -4*t1-1)^2*(3*t1^2+2*t1+1)^2)
cubic = EllipticCurve2x4(y^2 - x^3 - A*x^2 - B*x)

# Construct the linearly independent points on the elliptic curve as in [19].
x1 = (9*t1^4 +28*t1^3 +18*t1^2 +4*t1+1)^2*(11*t1^4 +12*t1^3 +2*t1^2 -4*t1-1)^2*(69*t1^4 +148*t1^3 +78*t1^2 +4*t1+1)^2
x2 = (3*t1^2 +2*t1+1)*(9*t1^2 +14*t1+7)^2*(13*t1^2-2*t1-1)*(9*t1^4 +28*t1^3 +18*t1^2 +4*t1+1)*(11*t1^4 +12*t1^3 +2*t1^2 -4*t1-1)^2*(31*t1^4 +52*t1^3 +22*t1^2 -4*t1-1)
x3 = (3*t1^2 +2*t1+1)*(9*t1^2 +14*t1+7)^2*(13*t1^2-2*t1-1)*(9*t1^4 +28*t1^3 +18*t1^2 +4*t1+1)^2*(11*t1^4 +12*t1^3 +2*t1^2 -4*t1-1)*(69*t1^4 +148*t1^3 +78*t1^2 +4*t1+1)
x4 = -(3*t1^2 +2*t1+1)^2*(9*t1^2 +14*t1+7)^2*(11*t1^4 +12*t1^3 +2*t1^2 -4*t1-1)^2*(31*t1^4 +52*t1^3 +22*t1^2 -4*t1-1)^2
var('M')
y1 = solve(M^2 - x1^3 - A*x1^2 - B*x1, M)[0].rhs()
y2 = solve(M^2 - x2^3 - A*x2^2 - B*x2, M)[0].rhs()
y3 = solve(M^2 - x3^3 - A*x3^2 - B*x3, M)[0].rhs()
y4 = solve(M^2 - x4^3 - A*x4^2 - B*x4, M)[0].rhs()
reset('M')
points = [(x1,y1),(x2,y2),(x3,y3),(x4,y4)]

# Compute the short Weierstrass form of the elliptic curve family and the linearly independent points.
a4, a6, weierstrass_points = compute_weierstrass_form(cubic, points)

# Uncomment this block to print 'a4', 'a6', and the entries in 'weierstrass_points'.
# print("a4 = " + str(a4))
# print("")
# print("a6 = " + str(a6))
# print("")
# for i in range(len(weierstrass_points)):
#     print("weierstrass_points[" + str(i) + "] = " + str(weierstrass_points[i]))
#     print("")

# Compute a 'ClassNumberBoundFamily' instance for this family, which can be used to obtain explicit numerical
# lower bounds for class numbers.
class_number_bound_family2x4 = compute_class_number_lower_bound_family(a4, a6, weierstrass_points, 8)

# Uncomment this block and replace 't1' with an integral constant to print the class number lower bound
# obtained from the elliptic curve corresponding to the specified parameters.
# class_number_bound2x4 = class_number_bound_family2x4.specialize([t1])
# print(str(class_number_bound2x4))

In [ ]:
#
# This cell contains the example we give in our paper for torsion structure Z/2Z x Z/6Z. The cubic defining the
# elliptic curve as well as the formulas for the linearly independent points are taken from citation [7]
# in our paper:
# A. Dujella and J.C. Peral, High-rank elliptic curves with torsion Z/2Z×Z/4Z induced by Diophantine triples, LMS J. Comput. Math. (1)17(2014), 282-288.
#

# Type definitions used for this example.
Polynomial2x6.<t1> = PolynomialRing(ZZ)
RationalFunction2x6 = Polynomial2x6.fraction_field()
EllipticCurve2x6.<x, y> = PolynomialRing(RationalFunction2x6)

# Construct the cubic that defines this elliptic curve family as in [19].
A = Polynomial2x6(- 2*(5764801 + 6588344*t1 - 21647416*t1^2 + 29445864*t1^3 - 9604*t1^4+ 27969592*t1^5 - 44631944*t1^6 + 9779112*t1^7 + 5909830*t1^8 - 1397016*t1^9 - 910856*t1^10 - 81544*t1^11 - 4*t1^12 - 1752*t1^13 - 184*t1^14 - 8*t1^15 + t1^16))
B = Polynomial2x6((-7 -10*t1 + t1^2)^3*(-7 + 2*t1 + t1^2)^3*(49 + 140*t1 - 106*t1^2 - 20*t1^3 + t1^4)*(49-28*t1+38*t1^2 +4*t1^3 +t1^4)^3*(49-112*t1+110*t1^2 +16*t1^3 +t1^4))
cubic = EllipticCurve2x6(y^2 - x^3 - A*x^2 - B*x)

# Construct the linearly independent points on the elliptic curve as in [19].
x1 = (49+140*t1 -106*t1^2 -20*t1^3 +t1^4)*(49+14*t1 +2*t1^2 -2*t1^3 + t1^4)^2*(49-112*t1+110*t1^2 +16*t1^3 +t1^4)
x2 = (-7-10*t1+t1^2)^2*(-7+2*t1+t1^2)^2*(49-28*t1+38*t1^2 +4*t1^3 +t1^4)^3/((-7-2*t1+t1^2)^2)
var('M')
y1 = solve(M^2 - x1^3 - A*x1^2 - B*x1, M)[0].rhs()
y2 = solve(M^2 - x2^3 - A*x2^2 - B*x2, M)[0].rhs()
reset('M')
points = [(x1,y1),(x2,y2)]

# Compute the short Weierstrass form of the elliptic curve family and the linearly independent points.
a4, a6, weierstrass_points = compute_weierstrass_form(cubic, points)

# Uncomment this block to print 'a4', 'a6', and the entries in 'weierstrass_points'.
# print("a4 = " + str(a4))
# print("")
# print("a6 = " + str(a6))
# print("")
# for i in range(len(weierstrass_points)):
#     print("weierstrass_points[" + str(i) + "] = " + str(weierstrass_points[i]))
#     print("")

# Compute a 'ClassNumberBoundFamily' instance for this family, which can be used to obtain explicit numerical
# lower bounds for class numbers.
class_number_bound_family2x6 = compute_class_number_lower_bound_family(a4, a6, weierstrass_points, 12)

# Uncomment this block and replace 't1' with an integral constant to print the class number lower bound
# obtained from the elliptic curve corresponding to the specified parameters.
# class_number_bound2x6 = class_number_bound_family2x6.specialize([t1])
# print(str(class_number_bound2x6))